In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

from sys import path
from os.path import abspath
path.append(abspath("/home/ng136/nico"))
import ng_functions as ng

from copy import deepcopy
from glob import glob
from pathlib import Path
import warnings
from time import time
from tqdm import tqdm

from datetime import datetime

import scvelo as scv

In [2]:
print(datetime.today())

2022-06-28 15:24:10.338415


# Useful dictionaries

In [3]:
n_states_dict = {'N1a' : 'Sell+Ngp+ neutrophils',
                 'N1b' : 'Sell+Lst1+ neutrophils',
                 'N2' : 'Sell+Cxcl10+ neutrophils',
                 'N3' : 'Cxcl3+ neutrophils',
                 'N4' : 'Siglecf+Xbp1+ neutrophils',
                 'N5' : 'Siglecf+Ccl3+ neutrophils',
                 'N6' : 'Siglecf+Ngp+ neutrophils'}

neu_palette_dict = {'Sell+Ngp+ neutrophils': '#6EA6CD',
                    'Sell+Lst1+ neutrophils': '#4A7BB7',
                    'Sell+Cxcl10+ neutrophils': '#364B9A',
                    'Cxcl3+ neutrophils': '#EAECCC',
                    'Siglecf+Xbp1+ neutrophils': '#A50026',
                    'Siglecf+Ccl3+ neutrophils': '#DD3D2D',
                    'Siglecf+Ngp+ neutrophils': '#F67E4B'}

In [4]:
exp3_samples = ['1A', '1B', '2A', '2B']
n_states = ['N1a', 'N1b', 'N2', 'N3', 'N4', 'N5', 'N6']

sample_dict = {'1A' : '1.KP19_CXCL10pos',
               '1B' : '1.KP19_CXCL10neg',
               '2A' : '2.KP19_CXCL10pos',
               '2B' : '2.KP19_CXCL10neg'}

lane_per_sample = {'1A': '1',
                   '1B': '1',
                   '2A': '2',
                   '2B': '2'}

# Import embedded data

In [5]:
adata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_neutrophils_acd40_annotated_embedding_25210x17824_backup_220422_15h46.h5ad')

# Import loom files from each

In [6]:
vdict = {sample: scv.read(f'/n/groups/klein/nico/neutrophils/data/velocyto/neutrophils_totalseq_aCD40/{sample}/{sample}.loom') for sample in exp3_samples}

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [7]:
for sample_id in exp3_samples:
    print(sample_id)
    for library_name in ['1.KP19_CXCL10pos', '2.KP19_CXCL10pos', '1.KP19_CXCL10neg', '2.KP19_CXCL10neg']:
        if adata[adata.obs['library_name'] == library_name].obs['barcode'].isin(vdict[sample_id].obs_names.map(lambda x : x.split(':')[-1][:-1] + '-1')).mean() == 1:
            print(library_name)

1A
1.KP19_CXCL10pos
1B
1.KP19_CXCL10neg
2A
2.KP19_CXCL10pos
2B
2.KP19_CXCL10neg


In [8]:
for k,v in vdict.items():
    v.var_names_make_unique()
    v.obs['sampleID'] = v.obs['library_name'] = v.obs_names.map(lambda x : x.split(':')[0]).map(sample_dict)
    v.obs['barcode'] = v.obs_names.map(lambda x : x.split(':')[-1][:-1] + '-1')
    v.obs['cellID'] = v.obs.merge(adata.obs, left_on=['library_name', 'barcode'], 
                                  right_on=['library_name', 'barcode'], how='left')['cellID'].values
    
    v.obs_names = v.obs['cellID'].rename(None)
    vdict[k] = v[~v.obs['cellID'].isna()]

# Merge and preprocess together

In [9]:
vdata = sc.concat(vdict)

In [10]:
inherited_umap = pd.DataFrame(adata.obsm['X_umap'], index=adata.obs_names, columns=['X_umap1', 'X_umap2'])
vdata.obsm['X_umap'] = vdata.obs.join(inherited_umap).iloc[:,-2:].values
vdata.obs['smoothed_Zilionis'] = adata.obs['smoothed_Zilionis']

vdata = vdata[vdata.obs['smoothed_Zilionis'].isin(n_states)]

vdata.obs['neutrophil_subset'] = vdata.obs['smoothed_Zilionis'].map(n_states_dict).astype('category')
vdata.obs['neutrophil_subset'] = vdata.obs['neutrophil_subset'].cat.reorder_categories(sorted(vdata.obs['neutrophil_subset'].cat.categories))

vdata.uns['neutrophil_subset_colors'] = vdata.obs['neutrophil_subset'].cat.categories.map(neu_palette_dict)

Trying to set attribute `.obs` of view, copying.


In [11]:
scv.pp.filter_and_normalize(vdata)


Normalized count data: X, spliced, unspliced.
Logarithmized X.


In [12]:
scv.pp.moments(vdata)


computing neighbors
    finished (0:00:35) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:38) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [ ]:
scv.tl.recover_dynamics(vdata, n_jobs=4)

recovering dynamics (using 4/32 cores)


  0%|          | 0/3069 [00:00<?, ?gene/s]

In [ ]:
scv.tl.velocity(vdata, mode='dynamical')

In [ ]:
scv.tl.velocity_graph(vdata, n_jobs=4)

# Save

In [19]:
start = time()
fname = '/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_neutrophils_scvelo_{}x{}_backup_{}.h5ad'.format(*vdata.shape,ng.now())
vdata.write(fname)
print(fname)
ng.print_etime(start)

... storing 'barcode' as categorical


/n/groups/klein/nico/neutrophils/backups/totalseq_exp3_neutrophils_scvelo_25210x32285_backup_220607_01h35.h5ad
Elapsed time: 0 minutes and 49.9 seconds.
